In [1]:
import pickle
import numpy as np
import pandas as pd
from ppt.metric import ACF_COS

In [2]:
original_data = pickle.load(open('./dataset/CharacterTrajectories/test_dict.pkl', 'rb'))
permuted_data = pickle.load(open('./dataset/CharacterTrajectories/permute_testdata/random_patchlen_006_permf_20.pkl', 'rb'))
label_data = pd.read_csv('./dataset/CharacterTrajectories/test_label.csv')

original_data = np.array([original_data[i] for i in range(len(original_data))])
permuted_data = np.array([permuted_data[i] for i in range(len(permuted_data))])
label_data = np.array([label_data['y_true'][i] for i in range(len(label_data))])
unique_labels = np.unique(label_data)

original_data = original_data.transpose(0, 2, 1)
permuted_data = permuted_data.transpose(0, 2, 1)

seq_len = original_data.shape[2]
channel_num = original_data.shape[1]

data_by_class_original = []
data_by_class_permuted = []
for label in unique_labels:
    data_by_class_original.append(original_data[label_data == label])
    data_by_class_permuted.append(permuted_data[label_data == label])

In [3]:
patch_len = 6
permute_freq = 20
num_classes = len(unique_labels)

metric = ACF_COS(
    channel_num=channel_num,
    original_time_len=seq_len,
    patch_len=patch_len,
    permute_freq=permute_freq,
    device='cuda',
    permute_tensor_size=1000
)

cos_acf = []
for cls_idx in range(num_classes):
    original_data = data_by_class_original[cls_idx]
    permuted_data = data_by_class_permuted[cls_idx]
    cos_acf.append(metric(original_data, permuted_data))
    
cos_acf = np.array(cos_acf).reshape(-1)
print(cos_acf)

Creating permutation tensor at shuffled_index/shuffled_index_random_20_6_03_20.pt, with strategy random, frequency 20, and patch length 6


Creating permutation tensor:   0%|          | 0/1000 [00:00<?, ?it/s]

[0.45939486 0.41904681 0.39361454 0.33518599 0.41883583 0.4127719
 0.39239213 0.45132259 0.47610439 0.40979294 0.39475422 0.41780006
 0.40117813 0.34373127 0.34470616 0.31646618 0.36175327 0.3609378
 0.23663316 0.44181572 0.41301472 0.37469869 0.46170162 0.4386347
 0.32444096 0.44141559 0.43694    0.27471586 0.34725262 0.38675748
 0.29391873 0.36549423 0.44538049 0.35116011 0.40549316 0.4155833
 0.38632043 0.27934705 0.35071761 0.43257881 0.48433175 0.50031521
 0.47865438 0.43421165 0.41488106 0.42688973 0.47248195 0.47927148
 0.23135981 0.37744567 0.39294316 0.36617058 0.37186893 0.39688449
 0.42480537 0.5057291  0.50117947 0.26771448 0.34183257 0.38267373]


In [4]:
mean = np.mean(cos_acf)
std = np.std(cos_acf)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: 0.39442414469005743
Std: 0.06330579523058594
